In [57]:
from mergedeep import merge

a = {
    'foo': {
        'bar': 'baz',
        'glarch': 'glarch',
    },
    'bar': 3
}

b = {
    'foo': {
        'bar': 'bar',
    },
    'baz': 'baz',
    'bar': {
        'blah': 'blah'
    }
}

pprint(merge({}, a, b))


{'bar': {'blah': 'blah'},
 'baz': 'baz',
 'foo': {'bar': 'bar', 'glarch': 'glarch'}}


In [67]:
import json
from legiscan import legiscan_api
import requests

@legiscan_api
def do_search(state: str, term: str, api_key: str):
    short_url = 'https://api.legiscan.com/'
    assemble_url = f'https://api.legiscan.com/?key={api_key}&op=getSearch&state={state}&query={term.replace(" ", "+")}'
    assemble_params = {
        'key': api_key,
        'op': 'getSearch',
        'state': state,
        'query': term,
    }
    return requests.get(short_url, params=assemble_params).text

json.loads(do_search('ID', '63')) # needs to be exact or the search can't find it :/

{'status': 'OK',
 'searchresult': {'summary': {'page': '1 of 2',
   'range': '1 - 50',
   'relevancy': '100% - 95%',
   'count': 84,
   'page_current': 1,
   'page_total': 2,
   'query': '(63:(pos=1))'},
  '0': {'relevance': 100,
   'state': 'ID',
   'bill_number': 'S1031',
   'bill_id': 1688658,
   'change_hash': '4a4e38a9512c06783b442fd51c648308',
   'url': 'https://legiscan.com/ID/bill/S1031/2023',
   'text_url': 'https://legiscan.com/ID/text/S1031/2023',
   'research_url': 'https://legiscan.com/ID/research/S1031/2023',
   'last_action_date': '2023-02-16',
   'last_action': 'Read First Time, Referred to Revenue & Taxation',
   'title': 'Amends existing law to provide that certain water rights and property of irrigation and canal companies exempt from taxation do not require certain approval.'},
  '1': {'relevance': 99,
   'state': 'ID',
   'bill_number': 'S1111',
   'bill_id': 1718408,
   'change_hash': '30634738b92d0e149232e9ea927123df',
   'url': 'https://legiscan.com/ID/bill/S111

In [2]:
from bs4 import BeautifulSoup as Soup
from itertools import islice
from nltk import download as nltk_download, pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string

nltk_download('punkt')
nltk_download('averaged_perceptron_tagger')
nltk_download('wordnet')

lem = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def extract_html_file(file_path: str):
    soup = None
    with open(file_path, 'r') as f:
        soup = Soup(f, 'html.parser')

    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    text = soup.get_text()
    tokens = (token for token in word_tokenize(text) if token not in string.punctuation)
    return list(tokens)

[(orig, lem) for orig, lem in ((word, lem.lemmatize(word, get_wordnet_pos(pos))) for word, pos in pos_tag(extract_html_file('bills/FL_S0254.html'))) if orig != lem]

[nltk_data] Downloading package punkt to /Users/amy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/amy/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/amy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[('entitled', 'entitle'),
 ('relating', 'relate'),
 ('treatments', 'treatment'),
 ('amending', 'amend'),
 ('granting', 'grant'),
 ('courts', 'court'),
 ('children', 'child'),
 ('are', 'be'),
 ('are', 'be'),
 ('being', 'be'),
 ('subjected', 'subject'),
 ('prescriptions', 'prescription'),
 ('procedures', 'procedure'),
 ('amending', 'amend'),
 ('requiring', 'require'),
 ('determining', 'determine'),
 ('is', 'be'),
 ('proceedings', 'proceeding'),
 ('amending', 'amend'),
 ('requiring', 'require'),
 ('courts', 'court'),
 ('as', 'a'),
 ('purposes', 'purpose'),
 ('determining', 'determine'),
 ('proceedings', 'proceeding'),
 ('prohibiting', 'prohibit'),
 ('treating', 'treat'),
 ('as', 'a'),
 ('circumstances', 'circumstance'),
 ('amending', 'amend'),
 ('defining', 'define'),
 ('purposes', 'purpose'),
 ('warrants', 'warrant'),
 ('proceedings', 'proceeding'),
 ('amending', 'amend'),
 ('providing', 'provide'),
 ('courts', 'court'),
 ('determinations', 'determination'),
 ('made', 'make'),
 ('circums

In [3]:
from bs4 import BeautifulSoup as Soup
from itertools import islice
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from pprint import pprint
import string

nltk.download('punkt')

def extract_html_file(file_path: str):
    soup = None
    with open(file_path, 'r') as f:
        soup = Soup(f, 'html.parser')

    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    text = soup.get_text()
    tokens = (token for token in word_tokenize(text) if token not in string.punctuation)
    return tokens
    # break into lines and remove leading and trailing space on each
    #lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    #chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    #text = "\n".join(chunk for chunk in chunks if chunk)

    #return text

ps = PorterStemmer()

pprint([(word, ps.stem(word)) for word in islice(extract_html_file('bills/FL_S0254.html'), 100)])


[('Florida', 'florida'),
 ('Senate', 'senat'),
 ('2023', '2023'),
 ('SB', 'sb'),
 ('254', '254'),
 ('By', 'by'),
 ('Senator', 'senat'),
 ('Yarborough', 'yarborough'),
 ('4-01859G-23', '4-01859g-23'),
 ('2023254__', '2023254__'),
 ('1', '1'),
 ('A', 'a'),
 ('bill', 'bill'),
 ('to', 'to'),
 ('be', 'be'),
 ('entitled', 'entitl'),
 ('2', '2'),
 ('An', 'an'),
 ('act', 'act'),
 ('relating', 'relat'),
 ('to', 'to'),
 ('treatments', 'treatment'),
 ('for', 'for'),
 ('sex', 'sex'),
 ('reassignment', 'reassign'),
 ('3', '3'),
 ('amending', 'amend'),
 ('s.', 's.'),
 ('61.517', '61.517'),
 ('F.S', 'f.'),
 ('granting', 'grant'),
 ('courts', 'court'),
 ('of', 'of'),
 ('this', 'thi'),
 ('4', '4'),
 ('state', 'state'),
 ('temporary', 'temporari'),
 ('emergency', 'emerg'),
 ('jurisdiction', 'jurisdict'),
 ('over', 'over'),
 ('children', 'children'),
 ('5', '5'),
 ('present', 'present'),
 ('in', 'in'),
 ('this', 'thi'),
 ('state', 'state'),
 ('if', 'if'),
 ('they', 'they'),
 ('are', 'are'),
 ('at', 'at')

[nltk_data] Downloading package punkt to /Users/amy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# import base64
import json
from pprint import pprint

filename = 'tmp/bill_meta_TX_HB1532.json'

result = None
with open(filename, 'r') as f:
    result = json.load(f)['bill']

extra = result['texts'][0].copy()
extra['date'] = '2023-01-01'
result['texts'].append(extra)
# pprint(result['texts'])
s = sorted(result['texts'], key=lambda x: x['date'], reverse=True)
doc_id = result['texts'][0]['doc_id']
doc_id

2658134

In [5]:
from legiscan import legiscan_api
import pandas as pd

raw = pd.read_json('tracktranslegislation.json')
sample = raw.sample(n=3, random_state=1234)

@legiscan_api
def get_bill_meta(legiscan_bill_id: str, api_key: str):
    assembled_url = f'https://api.legiscan.com/?key={api_key}&op=getBill&id={legiscan_bill_id}'
    resp = requests.get(assembled_url)

    if resp.ok:
        parsed = json.loads(resp.text)
        if parsed['status'] == 'ERROR':
            print(parsed['alert']['message'])
            return None
        return resp
    else:
        print(resp.status_code)
        return None
    
for idx, row in sample.iterrows():
    local_filename = os.path.join(
        'tmp',
        '_'.join([
            'bill',
            'meta',
            row["state"],
            *row["billId"].split(' '),
        ])
    ) + '.json'
    
    if os.path.exists(local_filename):
        print(f'skipping {local_filename}')
        continue
    
    resp = None
    resp = get_bill_meta(row['legiscanId'])
    if not resp:
        print(f'Could not download {local_filename}')
        continue
    
    print(f'got {local_filename}')
    with open(local_filename, 'wb') as f:
        f.write(resp.content)


NameError: name 'os' is not defined

In [ ]:
raw = pd.read_json('tracktranslegislation.json')
sample = raw.sample(n=3, random_state=1234)
sample

# 7 has wrong legiscanId, should be 2721785 i think
# 44 also has wrong legiscanId

In [ ]:
import base64
import json
from pprint import pprint

filename = 'bills/TX_HB1532'

result = None
with open(filename, 'r') as f:
    result = json.load(f)['text']

doc = result['doc']
extension = result['mime'].split('/')[-1]
new_filename = '.'.join([filename, extension])

with open(new_filename, 'wb') as f:
    f.write(base64.b64decode(doc))


In [ ]:
import json
from legiscan import legiscan_api
import os
import pandas as pd
from pprint import pprint
import requests

@legiscan_api
def get_bill_text(legiscan_bill_id: str, api_key: str):
    # https://api.legiscan.com/?key=5f61f50916512f9f21500f38877c22f7&op=getBillText&id=2736883
    assembled_url = f'https://api.legiscan.com/?key={api_key}&op=getBillText&id={legiscan_bill_id}'
    resp = requests.get(assembled_url)

    if resp.ok:
        parsed = json.loads(resp.text)
        if parsed['status'] == 'ERROR':
            print(parsed['alert']['message'])
            return None
        return resp
    else:
        print(resp.status_code)
        return None

raw = pd.read_json('tracktranslegislation.json')
sample = raw.sample(n=3, random_state=1234)

for idx, row in sample.iterrows():
    local_filename = os.path.join(
        'bills',
        '_'.join([
            row["state"],
            *row["billId"].split(' '),
        ])
    )
    
    if os.path.exists(local_filename):
        print(f'skipping {local_filename}')
    
    resp = get_bill_text(row['legiscanId'])
    if not resp:
        print(f'Could not download {local_filename}')
        continue
    
    with open(local_filename, 'wb') as f:
        f.write(resp.content)


In [ ]:
from functools import wraps
from legiscan import legiscan_api
import os

@legiscan_api
def sample_api_action(api_key: str):
    print(f'api key is {api_key}')

@legiscan_api
def get_bill_text(legiscan_bill_id: str, api_key: str):
    # https://api.legiscan.com/?key=5f61f50916512f9f21500f38877c22f7&op=getBillText&id=2736883
    assembled_url = f'https://api.legiscan.com/?key={api_key}&op=getBillText&id={legiscan_bill_id}'
    print(assembled_url)
    
sample_api_action(api_key='foo')


get_bill_text('12345')

In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.parse

host = 'https://www.house.mo.gov'
# url = f'{host}/BillContent.aspx?bill=HB1258&year=2023&code=R'
url = urllib.parse.urljoin(host, 'BillContent.aspx?bill=HB1258&year=2023&code=R')
page = requests.get(url)

# print(page.text)
soup = BeautifulSoup(page.content)
urllib.parse.urljoin(
    host, 
    soup.find_all(class_='textType')[0].find('a')['href'],
)


In [ ]:
import pandas as pd
import requests
from typing import Optional
import urllib.parse

def test_url(url: str, parent_id: str, anchor_index: int):
    page = requests.get(url)
    soup = BeautifulSoup(page.content)
    container = soup.find(id=parent_id)
    if not container:
        return None
    
    try:
        return container.find_all('a')[anchor_index]['href']
    except:
        return None

def prepare_url(relative_url: Optional[str]):
    if not relative_url:
        return 'NO RESULT'
    
    return urllib.parse.urljoin('https://legiscan.com/', relative_url)

def process_as_bill(frame) -> Optional[str]:
    return process_bill_link(frame['billLink'])

def process_bill_link(url: str) -> Optional[str]:
    print(url)
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='bill-last-action')
        anchors = container.find_all('a')
        href = anchors[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

def process_as_text(frame) -> Optional[str]:
    state = frame['state']
    bill_id = frame['billId'].replace(' ', '')
    year = row['billLink'].split('/')[-1] # don't trust statusDate
    text_link = f'https://legiscan.com/{state}/text/{bill_id}/{year}'
    return process_text_link(text_link)

def process_text_link(url: str) -> Optional[str]:
    print(url)
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='gaits-wrapper')
        href = container.find_all('a')[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

def process_as_draft(frame) -> Optional[str]:
    state = frame['state']
    bill_id = frame['billId'].replace(' ', '')
    year = row['billLink'].split('/')[-1] # don't trust statusDate
    text_link = f'https://legiscan.com/{state}/drafts/{bill_id}/{year}'
    return process_text_link(text_link)
    
def process_draft_link(url: str) -> Optional[str]:
    print(url)
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='gaits-wrapper')
        href = container.find_all('a')[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

raw = pd.read_json('tracktranslegislation.json')
sample = raw.copy()
# sample = raw.sample(n=20, random_state=2)
# sample = raw.loc[raw.state == 'AR']
# sample = raw.loc[0:20]
# print(sample)

for idx, row in sample.iterrows():
    continue
    bill_id = ' '.join([row['state'], row['billId']])
#    year = row['billLink'].split('/')[-1]
    
#    bill_link = row['billLink']
#    draft_link = f'https://legiscan.com/{row["state"]}/drafts/{row["billId"].replace(" ", "")}/{year}' # https://legiscan.com/AZ/drafts/HB2517/2023
#    text_link = f'https://legiscan.com/{row["state"]}/text/{row["billId"].replace(" ", "")}/{year}'
#    comments_link = f'https://legiscan.com/{row["state"]}/comments/{row["billId"].replace(" ", "")}/{year}'
    
    searches = [
        process_as_bill,
        process_as_text,
    ]

    print(f'{bill_id}')
    for search in searches:
        print(search(row))

    print()


In [ ]:
# https://legiscan.com/TX/comments/HB1029/2023

from bs4 import BeautifulSoup
from pprint import pprint
import requests
from typing import Optional
import urllib.parse

def process_as_bill(frame) -> Optional[str]:
    return process_bill_link(frame['billLink'])

def process_bill_link(url: str) -> Optional[str]:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='bill-last-action')
        anchors = container.find_all('a')
        href = anchors[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

def process_as_text(frame) -> Optional[str]:
    state = frame['state']
    bill_id = frame['billId'].replace(' ', '')
    year = row['billLink'].split('/')[-1] # don't trust statusDate
    text_link = f'https://legiscan.com/{state}/text/{bill_id}/{year}'
    return process_text_link(text_link)

def process_text_link(url: str) -> Optional[str]:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='gaits-wrapper')
        href = container.find_all('a')[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

def process_as_draft(frame) -> Optional[str]:
    state = frame['state']
    bill_id = frame['billId'].replace(' ', '')
    year = row['billLink'].split('/')[-1] # don't trust statusDate
    text_link = f'https://legiscan.com/{state}/drafts/{bill_id}/{year}'
    return process_text_link(text_link)
    
def process_draft_link(url: str) -> Optional[str]:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='gaits-wrapper')
        href = container.find_all('a')[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

    
#text_link = 'https://legiscan.com/TX/text/HB3147/2023'
#print(text_link)
#print(process_text_link(text_link))

#bill_link = 'https://legiscan.com/TX/bill/HB976/2023'
#print(bill_link)
#print(process_bill_link(bill_link))
#print(process_as_bill(sample.loc[303]))

print(process_as_bill(raw.loc[251]))
print(process_as_text(raw.loc[251]))
print(process_as_draft(raw.loc[251]))

#host = 'https://www.house.mo.gov'
# url = f'{host}/BillContent.aspx?bill=HB1258&year=2023&code=R'
#url = 'https://legiscan.com/SD/text/HB1080/2023' #urllib.parse.urljoin(host, 'BillContent.aspx?bill=HB1258&year=2023&code=R')
#page = requests.get(url)

# print(page.text)
#soup = BeautifulSoup(page.content)
#container = soup.find(id='gaits-wrapper')
#pprint(container.find_all('a'))
#urllib.parse.urljoin(
#    host, 
#    soup.find_all(class_='textType')[0].find('a')['href'],
#)